<a href="https://colab.research.google.com/github/sainisatish/DeepLearningFromScratch/blob/main/Cat_And_Dog_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files

In [2]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"satishsaini","key":"6f61f9669d77c8a5654931b01aefdc8d"}'}

In [3]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle competitions download -c dogs-vs-cats

100% 809M/812M [00:21<00:00, 38.5MB/s]
100% 812M/812M [00:21<00:00, 39.4MB/s]


In [5]:
!unzip -qq dogs-vs-cats.zip

In [6]:
!unzip -qq train.zip

In [7]:
import os,shutil,pathlib

In [8]:
original_dir = pathlib.Path("train")
new_base_dir = pathlib.Path("cats_vs_dogs_small")

In [10]:
def make_subset(subset_name, start_index, end_index): 
    for category in ("cat", "dog"):
        dir = new_base_dir / subset_name / category
        os.makedirs(dir)
        fnames = [f"{category}.{i}.jpg" 
                  for i in range(start_index, end_index)]
        for fname in fnames:
            shutil.copyfile(src=original_dir / fname,
                            dst=dir / fname)

In [11]:
make_subset("train", start_index=0, end_index=1000)
make_subset("validation", start_index=1000, end_index=1500) 
make_subset("test", start_index=1500, end_index=2500)

In [12]:
from tensorflow import keras
from tensorflow.keras import layers

In [13]:
inputs = keras.Input(shape=(180,180,3))
x=layers.Rescaling(1./255)(inputs)
x=layers.Conv2D(filters=32,kernel_size=3,activation='relu')(x)
x=layers.MaxPooling2D(pool_size=2)(x)
x=layers.Conv2D(filters=64,kernel_size=3,activation='relu')(x)
x=layers.MaxPooling2D(pool_size=2)(x)
x=layers.Conv2D(filters=128,kernel_size=3,activation='relu')(x)
x=layers.MaxPooling2D(pool_size=2)(x)
x=layers.Conv2D(filters=256,kernel_size=3,activation='relu')(x)
x=layers.MaxPooling2D(pool_size=2)(x)
x=layers.Conv2D(filters=256,kernel_size=3,activation='relu')(x)
x=layers.Flatten()(x)
outputs=layers.Dense(1,activation='sigmoid')(x)
model = keras.Model(inputs=inputs,outputs=outputs)



In [14]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 180, 180, 3)]     0         
                                                                 
 rescaling (Rescaling)       (None, 180, 180, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 178, 178, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 89, 89, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 87, 87, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 43, 43, 64)       0         
 2D)                                                         

In [15]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics='accuracy')

In [16]:
from tensorflow.keras.utils import image_dataset_from_directory 

In [21]:
train_dataset=image_dataset_from_directory(new_base_dir/"train",
                                           image_size=(180,180),
                                           batch_size=32)

validation_dataset = image_dataset_from_directory(new_base_dir /"validation",
                                                  image_size=(180,180),
                                                  batch_size=32)

test_size = image_dataset_from_directory(new_base_dir /"test",
                                         image_size=(180,180),
                                         batch_size=32)

Found 2000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
Found 2000 files belonging to 2 classes.


In [23]:
for data_batch,label_batch in train_dataset:
    print(f"data batch shape {data_batch.shape}")
    print(f"label batch shape { label_batch.shape}")
    break

data batch shape (32, 180, 180, 3)
label batch shape (32,)


In [24]:
callbacks = [keras.callbacks.ModelCheckpoint(filepath="convnet_from_scratch.keras",save_best_only=True,monitor="val_loss")]

In [ ]:
history = model.fit(train_dataset,epochs=30,validation_data=validation_dataset,callbacks=callbacks)

Epoch 1/30
63/63 [==============================] - 144s 2s/step - loss: 0.6982 - accuracy: 0.4990 - val_loss: 0.6930 - val_accuracy: 0.5000
Epoch 2/30
63/63 [==============================] - 134s 2s/step - loss: 0.6929 - accuracy: 0.5240 - val_loss: 0.6886 - val_accuracy: 0.5080
Epoch 3/30
63/63 [==============================] - 131s 2s/step - loss: 0.6712 - accuracy: 0.5940 - val_loss: 0.7775 - val_accuracy: 0.5230
Epoch 4/30
63/63 [==============================] - 132s 2s/step - loss: 0.6332 - accuracy: 0.6490 - val_loss: 0.6243 - val_accuracy: 0.6180
Epoch 5/30
63/63 [==============================] - 130s 2s/step - loss: 0.5951 - accuracy: 0.6860 - val_loss: 0.5813 - val_accuracy: 0.6800
Epoch 6/30
63/63 [==============================] - 135s 2s/step - loss: 0.5596 - accuracy: 0.7140 - val_loss: 0.5785 - val_accuracy: 0.6910
Epoch 7/30
63/63 [==============================] - 131s 2s/step - loss: 0.5048 - accuracy: 0.7565 - val_loss: 0.5172 - val_accuracy: 0.7380
Epoch 8/30
63